In [1]:
#from veneer.manage import create_command_line

#import veneer-py

from veneer.manage import start, create_command_line, kill_all_now
import veneer
import geopandas
import pandas as pd

In [2]:
veneer_install = 'D:/PythonScriptingForGBR/VeneerPlugin/v411'
source_version = '4.1.1'
cmd_directory = 'C:\\tempveneer\\veneer_cmd'
path = create_command_line(veneer_install,source_version,dest=cmd_directory)
path

['C:\\Program Files\\eWater\\Source 3.6.5.1497', 'C:\\Program Files\\eWater\\Source 3.8.10.2480 NightlyBuild', 'C:\\Program Files\\eWater\\Source 3.8.10.2553 Beta (public version)', 'C:\\Program Files\\eWater\\Source 3.8.10.2620 NightlyBuild', 'C:\\Program Files\\eWater\\Source 3.8.10.2635 Beta', 'C:\\Program Files\\eWater\\Source 3.8.10.2692 Beta', 'C:\\Program Files\\eWater\\Source 3.8.10.2764 Beta', 'C:\\Program Files\\eWater\\Source 3.8.10.2954 NightlyBuild', 'C:\\Program Files\\eWater\\Source 3.8.10.2983 Beta', 'C:\\Program Files\\eWater\\Source 3.8.10.3034 Beta', 'C:\\Program Files\\eWater\\Source 3.8.11.2833 Beta', 'C:\\Program Files\\eWater\\Source 3.8.15.3355 Beta', 'C:\\Program Files\\eWater\\Source 3.8.16.3538 Beta', 'C:\\Program Files\\eWater\\Source 3.8.17.3618 Beta', 'C:\\Program Files\\eWater\\Source 3.8.18.3689 Beta', 'C:\\Program Files\\eWater\\Source 3.8.18.3715 Beta', 'C:\\Program Files\\eWater\\Source 3.8.19.3836 Beta', 'C:\\Program Files\\eWater\\Source 3.8.21.4026

WindowsPath('C:/tempveneer/veneer_cmd/FlowMatters.Source.VeneerCmd.exe')

In [3]:
#catchment_project='D:\\Xike\\bu\\bu_reefhydro_bo_411_noMP.rsproj'
catchment_project='D:\\Fitzroy\\v411_projs\\FI_411_Unplugged.rsproj'
#catchment_project='D:\\Beckers\\Beckers_Simple_411.rsproj'
num_copies=1    # Important - set this to be a number ~ the number of CPU cores in your system!
first_port=9950

In [4]:
processes,ports = start(catchment_project,
                        n_instances=num_copies,
                        ports=first_port,
                        debug=True,
                        veneer_exe=path,
                        remote=False)

Starting C:\tempveneer\veneer_cmd\FlowMatters.Source.VeneerCmd.exe -p 9950 -s  D:\Fitzroy\v411_projs\FI_411_Unplugged.rsproj
[0] Loading plugins

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\DERMTools.dll

[0] Loaded D:\PythonScriptingForGBR\VeneerPlugin\v411\FlowMatters.Source.Veneer.dll

[0] Plugins loaded (2/2)

[0] Opening project file: D:\Fitzroy\v411_projs\FI_411_Unplugged.rsproj

ERROR[0] log4net:ERROR Failed to find configuration section 'log4net' in the application's .config file. Check your .config file for the <log4net> and <configSections> elements. The configuration section should look like: <section name="log4net" type="log4net.Config.Log4NetConfigurationSectionHandler,log4net" />

[0] Loading project

[0] Project Loaded

[0] Fitzroy Rebuild 2014

[0] Fitzroy Rebuild 2014

[0] [3:45:32 PM] Veneer, by Flow Matters: http://www.flowmatters.com.au

[0] [3:45:32 PM] Started Source RESTful Service on port:9950

Server 0 on port 9950 is ready
[0] Server started. 

In [5]:
vs = [veneer.Veneer(port=p) for p in ports]

In [6]:
#kill_all_now(processes)

In [7]:
%matplotlib inline
v = vs[0]
#v.network().as_dataframe().plot()

In [8]:
the_network = v.network()
outlets = the_network.outlet_nodes()
outlets

[{'geometry': {'coordinates': [1788493.63051, -2510245.4690071], 'type': 'Point'}, 'properties': {'schematic_location': [647.7285766601562, 184.92437744140625], 'icon': '/resources/ConfluenceNodeModel', 'name': 'Outlet Node1', 'feature_type': 'node'}, 'id': '/network/nodes/1', 'type': 'Feature'}, {'geometry': {'coordinates': [1787233.63051, -2513485.4690071], 'type': 'Point'}, 'properties': {'schematic_location': [645.1803588867188, 190.55810546875], 'icon': '/resources/ConfluenceNodeModel', 'name': 'Outlet Node2', 'feature_type': 'node'}, 'id': '/network/nodes/4', 'type': 'Feature'}, {'geometry': {'coordinates': [1782283.63051, -2526745.4690071], 'type': 'Point'}, 'properties': {'schematic_location': [635.1695556640625, 213.614990234375], 'icon': '/resources/ConfluenceNodeModel', 'name': 'Outlet Node3', 'feature_type': 'node'}, 'id': '/network/nodes/7', 'type': 'Feature'}, {'geometry': {'coordinates': [1789363.63051, -2531815.4690071], 'type': 'Point'}, 'properties': {'schematic_locat

In [9]:
sDate = v.model.get('scenario.CurrentConfiguration.StartDate')
print(sDate)

eDate = v.model.get('scenario.CurrentConfiguration.EndDate')
print(eDate)

#v.run_model()

1/01/2000 12:00:00 AM
20/07/2014 12:00:00 AM


In [10]:
def processLinks(theNetwork, allLinkAreas, currentLink, knownDownStreamLinksToUpdate):
    
    theLinkName = currentLink['properties']['name']
    theLinkID = currentLink['id']
    
    # I think this will actually return a catchment record, not just an ID
    theCat = theNetwork['features'].find_by_link(theLinkID)[0]
    
    #print(theCat)
    
    thisCatName = theCat['properties']['name']
    thisCatArea = theCat['properties']['areaInSquareMeters']
    
    #print("Processing " + thisCatName + " with area of " + str(thisCatArea))
    
    allLinkAreas[theLinkName]['area'] += thisCatArea
    allLinkAreas[theLinkName]['contlinks'] += 1
    
    newDSLinksToUpdate = []
    for eachDSLink in knownDownStreamLinksToUpdate:
        lName = str(eachDSLink)
        #print("DS Link " + lName + " with area of ")
        allLinkAreas[lName]['area'] += thisCatArea
        allLinkAreas[lName]['contlinks'] += 1
        newDSLinksToUpdate.append(lName)
    
    # now add the link we've just processed
    newDSLinksToUpdate.append(theLinkName)
    
    #print("Link info: " + str(currentLink))
    
    fNodeID = currentLink['properties']['from_node']
    
    #print("Node info: " + fNodeID)
    
    fNode = the_network['features'].find_by_id(fNodeID)
    #print(fNode)
    
    fNodeName = fNode[0]['properties']['name']
    #print(fNodeName)
    
    ## recurse
    #print(the_network.upstream_links(fNode[0]))
    for upLink in the_network.upstream_links(fNode[0]):
        #tNodeID = upLink['properties']['to_node']
        #tNode = the_network['features'].find_by_id(tNodeID)
        #print("Now processing: " + upLink['properties']['name'] + " upstream of " + tNode[0]['properties']['name'] + " with initial Region: " + currentReg)
        processLinks(theNetwork, allLinkAreas, upLink, newDSLinksToUpdate)

    return

In [11]:
noRegStr = 'NoReg'
##gaugeList = ['120219A','120206A','link for catchment SC #139']
##gaugeList = ['120205A','120206A','120207A','120209A','120212A','120213A','120214A','120219A']

# will assign regions to links, pair with subcatchs later
# Can it include a contributing link count?
all_links = the_network['features'].find_by_feature_type('link')
linkAreas = {}

## Populate the dictionary of relationships first
for link in all_links:
    #linkAreas[link['properties']['name']] = 0
    linkAreas[link['properties']['name']] = {}
    linkAreas[link['properties']['name']]['area'] = 0
    linkAreas[link['properties']['name']]['contlinks'] = 0

## now process the network
for outNode in outlets:
    curReg = noRegStr
    
    #print(outNode.keys())
    #print(outNode['properties'].keys())
    #print(outNode['properties']['name'])
    
    thisNodeName = outNode['properties']['name']
    
    #this will be an empty list, built up for each link as we get to it
    downStreamLinksToUpdate = []
    
    ## now we begin processing
    for upLink in the_network.upstream_links(outNode):
        processLinks(the_network, linkAreas, upLink, downStreamLinksToUpdate)
    

In [12]:
#linkAreas_df = pd.DataFrame(list(linkAreas.items()), columns=['LinkName','AreaM2'])

#orient=index gets our first keys as rows
linkAreas_df = pd.DataFrame.from_dict(linkAreas, orient='index')
linkAreas_df.columns=['LinkCount','AreaM2']
linkAreas_df.index.name = 'LinkName'
linkAreas_df.reset_index(inplace=True)
#linkAreas_df.reset_index()
#linkAreas_df.columns=['LinkCount','AreaM2']

#linkAreas_df.columns=['LinkName','LinkCount','AreaM2']
linkAreas_df
#help(v.network)
#help(run_results2)

,LinkName,LinkCount,AreaM2
0,GS130005A,1658,1.357281e+11
1,GS130104A,549,4.603197e+10
2,GS130106A,613,5.078977e+10
3,GS130109A,927,7.530460e+10
4,GS130315AC,10,6.012585e+08
5,GS130322A,449,4.020739e+10
6,Link #1,0,0.000000e+00
7,Link #10,0,0.000000e+00
8,Link #11,0,0.000000e+00
9,Link #12,0,0.000000e+00


In [13]:
network_df = v.network().as_dataframe()
catchdf = network_df[network_df.feature_type=='catchment']
linkdf = network_df[network_df.feature_type=='link']

catchlinkdf = pd.merge(catchdf, linkdf, how='inner', left_on=['link'], right_on=['id'])
catchlinkdf = catchlinkdf.rename(columns={'areaInSquareMeters_y':'linkarea', 'areaInSquareMeters_x':'areaInSquareMeters', 'feature_type_y':'link_feat_type', 'feature_type_x':'feature_type', 'geometry_y':'link_geom', 'geometry_x':'geometry', 'name_y': 'link_name'})

catchlinkdfregdf = pd.merge(catchlinkdf, linkAreas_df, how='inner', left_on=['link_name'], right_on=['LinkName'])
catchlinkdfregdf = catchlinkdfregdf.rename(columns={'name_x':'SubCat'})
catchlinkdfregdf.head(8)

,areaInSquareMeters,feature_type,from_node_x,geometry,icon_x,link_x,SubCat,schematic_location_x,to_node_x,id_x,...,link_geom,icon_y,link_y,link_name,schematic_location_y,to_node_y,id_y,LinkName,LinkCount,AreaM2
0,164339100.0,catchment,NaN,"(POLYGON ((1631008.63051 -2601880.4690071, 163...",NaN,/network/link/1607,SC #1608,NaN,NaN,/network/catchments/5810,...,"LINESTRING (1639363.63051 -2603035.4690071, 16...",NaN,NaN,link for catchment SC #1608,NaN,/network/nodes/3322,/network/link/1607,link for catchment SC #1608,7,773190000.0
1,53183700.0,catchment,NaN,"(POLYGON ((1654048.63051 -2586970.4690071, 165...",NaN,/network/link/1617,SC #1618,NaN,NaN,/network/catchments/5819,...,"LINESTRING (1657273.63051 -2597965.4690071, 16...",NaN,NaN,link for catchment SC #1618,NaN,/network/nodes/3342,/network/link/1617,link for catchment SC #1618,1,53183700.0
2,141467400.0,catchment,NaN,"(POLYGON ((1674538.63051 -2617480.4690071, 167...",NaN,/network/link/1627,SC #1628,NaN,NaN,/network/catchments/5828,...,"LINESTRING (1679173.63051 -2622775.4690071, 16...",NaN,NaN,link for catchment SC #1628,NaN,/network/nodes/3353,/network/link/1627,link for catchment SC #1628,1,141467400.0
3,587700.0,catchment,NaN,"(POLYGON ((1678078.63051 -2595220.4690071, 167...",NaN,/network/link/1637,SC #1638,NaN,NaN,/network/catchments/5837,...,"LINESTRING (1677523.63051 -2595355.4690071, 16...",NaN,NaN,link for catchment SC #1638,NaN,/network/nodes/3379,/network/link/1637,link for catchment SC #1638,3,333511200.0
4,134812800.0,catchment,NaN,"(POLYGON ((1709248.63051 -2572990.4690071, 170...",NaN,/network/link/1647,SC #1648,NaN,NaN,/network/catchments/5846,...,"LINESTRING (1708153.63051 -2580985.4690071, 17...",NaN,NaN,link for catchment SC #1648,NaN,/network/nodes/3394,/network/link/1647,link for catchment SC #1648,1,134812800.0
5,2019600.0,catchment,NaN,"(POLYGON ((1705198.63051 -2603380.4690071, 170...",NaN,/network/link/1657,SC #1658,NaN,NaN,/network/catchments/5855,...,"LINESTRING (1704673.63051 -2603725.4690071, 17...",NaN,NaN,link for catchment SC #1658,NaN,/network/nodes/3416,/network/link/1657,link for catchment SC #1658,3,142815600.0
6,151012800.0,catchment,NaN,"(POLYGON ((1741468.63051 -2576470.4690071, 174...",NaN,/network/link/1667,SC #1668,NaN,NaN,/network/catchments/5864,...,"LINESTRING (1744543.63051 -2583835.4690071, 17...",NaN,NaN,link for catchment SC #1668,NaN,/network/nodes/2170,/network/link/1667,link for catchment SC #1668,1,151012800.0
7,136793700.0,catchment,NaN,"(POLYGON ((1761058.63051 -2608870.4690071, 176...",NaN,/network/link/1677,SC #1678,NaN,NaN,/network/catchments/5873,...,"LINESTRING (1761823.63051 -2611675.4690071, 17...",NaN,NaN,link for catchment SC #1678,NaN,/network/nodes/3450,/network/link/1677,link for catchment SC #1678,1,136793700.0


In [14]:
catchlinkdfregdf.to_csv('D:/Fitzroy/v411_projs/FIUpstreamAreas.csv', columns=['SubCat', 'LinkName', 'AreaM2', 'LinkCount'], index=False)

In [17]:
kill_all_now(processes)